In [56]:
from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib inline


# II. Deuxième équation:   

### $$\partial^2_x f(x,y) + \partial^2_y f(x,y) - \gamma f(x,y) = 0 $$


Posons $\beta = h(\gamma)$.
Soit $(X_n)_{n \in \mathbb{N}}$ la marche aléatoire régulière en deux dimensions initialement en $(i,j)$. On considère la suite de fonctions $(v_n)_{n \in \mathbb{N}}$ définie par: 
### $$v_n(i) = \mathbb{E}_i\big[\phi(X_n) exp \big( - \beta \sum\limits^{n}_{j = 0} \mathbb{1}_{\{ X_j \notin \partial{\mathbb{D}} \}}\big)\big]$$

En conditionnant par rapport au premier pas, étant donné que la marche est symmétrique on obtient pour tout $(i,j) \notin \partial \mathbb{D}$:
 $v_{n+1}(i,j) = \frac {1} {4} \mathbb{E} \big[ \phi(X_{n+1})exp \big( - \beta \sum_\limits{i = 0}^{n} \mathbb{1}_{X_i \notin \{0;L \} }\big) | X_0 = (i,j), X_1 = (i+1, j)  \big]+ \frac {1} {4} \mathbb{E} \big[ \phi(X_{n+1})exp \big( - \beta \sum_\limits{i = 0}^{n} \mathbb{1}_{X_i \notin \{0;L \} }\big) | X_0 = (i,j), X_1 = (i-1, j)  \big]
   +
   \frac {1} {4} \mathbb{E} \big[ \phi(X_{n+1})exp \big( - \beta \sum_\limits{i = 0}^{n} \mathbb{1}_{X_i \notin \{0;L \} }\big) | X_0 = (i,j), X_1 = (i, j+1)  \big]
   +
   \frac {1} {4} \mathbb{E} \big[ \phi(X_{n+1})exp \big( - \beta \sum_\limits{i = 0}^{n} \mathbb{1}_{X_i \notin \{0;L \} }\big) | X_0 = (i,j), X_1 = (i, j-1)  \big]
   $
   

Puis (propriété de Markov), la suite $(X_{n+1})_{n \in \mathbb{N}}$ est une chaîne de Markov de même transition que $(X_{n})_{n \in \mathbb{N}}$, initialement en $X_1$. Par conséquent:
$$v_{n+1}(i,j) = \frac {e^{-\beta}} {4} [v_{n}(i+1,j) + v_{n}(i,j+1) + v_{n}(i-1,j) + v_{n}(i,j-1)] $$

 $$v_{n+1}(i,j) - v_{n}(i,j)  = \frac {e^{-\beta}} {4} [v_{n}(i+1,j) + v_{n}(i,j+1) + v_{n}(i-1,j) + v_{n}(i,j-1) - 4 v_{n}(i,j)] - (1 - e^{-\beta})v_n(i,j) $$


Notons $\tilde{\Delta}$ le laplacien discret, qui envoie toute fonction $f: \mathbb{D} \to \mathbb{R}$ sur la fonction $$\tilde{\Delta}f : (i,j) \mapsto \frac {[f(i+1,j) - f(i,j)] + [f(i, j+1)- f(i,j)] + [f(i-1, j) - f(i,j)] + [f(i, j-1) - f(i,j)]} {4} $$

On peut alors réécrire l'égalité précedente sous la forme:
 $$v_{n+1}(i,j) - v_{n}(i,j)  =  {e^{-\beta}} \tilde{\Delta}v_n(i,j)  - (1 - e^{-\beta})v_n(i,j) $$


$v_n$ vérifie la version discrétisée de l'équation:
$$\partial_t f(x,y,t) = e^{-\beta}\Delta f(x,y,t) - (1 - e^{-\beta}) f(x,y,t)$$


Pour $t \to \infty$ le régime stationnaire est atteint, auquel cas $\partial_t f(x,y,t) = 0$. rappelons qu'on a choisi $\beta$ de telle sorte que $\gamma = e^{\beta} - 1$. Ainsi pour $n$ assez grand $v_n(i)$ est proche de la valeur en $i$ de la solution de l'équation:
### $$\Delta f(x,y,t) - \gamma f(x,y,t) =0 $$


In [57]:
#! /usr/bin/env python

import random
import numpy as np
import time

In [58]:
# define phi as f is at time t = 0 - in particular, phi defines border conditions
def phi(x, y, L):
    if x == 0 or y == 0:
        return 2.
    return 0.
vphi = np.vectorize(phi)

In [59]:
def v_n(init, L, beta, n, K):
    somme = 0
    for _ in range(K):
        x = init[0]
        y = init[1]
        coordList = [[x,y]]
        number_non_touches = 0
        for j in range(n):
            coordList.append([x,y])
            if x != 0 and y != 0 and y != L and x != L: # in that case the walk's fate is sealed
                number_non_touches += 1
                rand = random.random()
                # Go with equal probability left, up, right or down
                if 0 <= rand < 0.25:
                    x -= 1
                elif 0.25 <= rand < 0.5:
                    x += 1
                elif 0.5 <= rand < 0.75:
                    y -= 1
                else:
                    y += 1
                
        somme += vphi(np.array([coord[0] for coord in coordList]),np.array([coord[1] for coord in coordList]), L)*np.exp(- beta*np.arange(n+1))
    return somme/K


In [60]:
K = 50
L = 40
N = 900
beta = 1 


In [61]:
arrayImages = np.zeros((N,L,L))
for i in range(L):
    for j in range(L):
        ls = v_n([i,j], L, beta, N, K)
        for n in range(N):
            arrayImages[n][i][j] = ls[n]

            


In [62]:
%matplotlib inline
import matplotlib.pyplot as plt
def _show(frame=widgets.IntSlider(min=0,max=len(arrayImages) - 1,step=3,value=0)):
    fig, ax = plt.subplots(figsize=(7,7))
    ax.imshow(arrayImages[frame][:][:].T, origin="lower",
               extent=[0, 1, 0, 1], cmap="jet")
interact(_show)

interactive(children=(IntSlider(value=0, description='frame', max=899, step=3), Output()), _dom_classes=('widg…

<function __main__._show(frame=IntSlider(value=0, description='frame', max=899, step=3))>